In [1]:
#importing necessary libraries
import pandas as pd
import numpy as np
import lxml
import requests

In [2]:
df=pd.read_html('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0, na_values='Not assigned')[0]
df.dropna(subset=['Borough'], inplace=True)
df.Neighbourhood.fillna(df.Borough, inplace=True)
df=df.groupby('Postcode', as_index=False).aggregate({'Borough':'first', 'Neighbourhood':', '.join})

In [3]:
gc=pd.read_csv('https://cocl.us/Geospatial_data')
gc.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df=pd.merge(df, gc, on='Postcode')
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
CLIENT_ID = 'DS0UKDSWSCFUFY0TV1YPIDYIEGUWQCSKZGHOIP45FLGZDC1X' # your Foursquare ID
CLIENT_SECRET = 'VYX0TO1JCGIJ4O3TSM3PW41M3XBUKGIESGRRJNZYYRXGHGZ4' # your Foursquare Secret
VERSION = '20181223' # Foursquare API version

In [5]:
latitude_tor = 43.6532
longitude_tor = -79.3832

LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_tor, longitude_tor, VERSION, radius, LIMIT)

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c1f7eb01ed2194bd23d7d75'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 125,
  'suggestedBounds': {'ne': {'lat': 43.6577000045, 'lng': -79.37699210971401},
   'sw': {'lat': 43.648699995499996, 'lng': -79.389407890286}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          '

In [7]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Eggspectation Bell Trinity Square,Breakfast Spot,43.653144,-79.381980
3,Indigo,Bookstore,43.653515,-79.380696
4,CF Toronto Eaton Centre,Shopping Mall,43.653817,-79.380650
5,Old City Hall,Monument / Landmark,43.652009,-79.381744
6,Bannock,Restaurant,43.652101,-79.381178
7,LUSH,Cosmetics Shop,43.653557,-79.380400
8,Apple Eaton Centre,Electronics Store,43.652823,-79.380615
9,Noodle King,Asian Restaurant,43.651706,-79.383046


In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

In [11]:
backup=toronto_venues

In [12]:
toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764042,-79.193371,Rental Car Location
7,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
8,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot
9,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop


In [13]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",27,27,27,27,27,27
Berczy Park,54,54,54,54,54,54


In [14]:
# one hot encoding
tor_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
# fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
# tor_onehot = tor_onehot[fixed_columns]
nbd=tor_onehot['Neighborhood']
tor_onehot.drop(['Neighborhood'], axis=1, inplace=True)
tor_onehot.insert(0, 'Neighborhood', nbd)

tor_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
tor_onehot.shape

(2247, 274)

In [16]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0


In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Bar,Gym,Bakery,Clothing Store,Hotel
1,Agincourt,Skating Rink,Clothing Store,Lounge,Breakfast Spot,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Fried Chicken Joint,Pharmacy,Coffee Shop,Sandwich Place,Beer Store,Fast Food Restaurant,Golf Course,Dance Studio
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Athletics & Sports,Pool,Pub,Sandwich Place,Skating Rink,Pharmacy,Gym,Gift Shop
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Pizza Place,Frozen Yogurt Shop,Shopping Mall,Fast Food Restaurant,Bridal Shop,Sandwich Place,Diner,Bank,Sushi Restaurant
6,Bayview Village,Bank,Café,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
7,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Thai Restaurant,Fast Food Restaurant,Sushi Restaurant,Grocery Store,Butcher,Restaurant,Café,Pub
8,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Steakhouse,Bakery,Italian Restaurant,Seafood Restaurant,Farmers Market,Pub,Cheese Shop
9,"Birch Cliff, Cliffside West",College Stadium,Café,General Entertainment,Skating Rink,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [19]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# scarb_data = scarborough_data.drop(16)
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 1, 0, 0, 2, 2, 2, 2, 2])

In [20]:
tor_merged = df

# # merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood', how='right')

# # add clustering labels
tor_merged['Cluster Labels'] = kmeans.labels_

tor_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Health & Beauty Service,2
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Golf Course,Bar,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio,2
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Pizza Place,Medical Center,Electronics Store,Mexican Restaurant,Breakfast Spot,Rental Car Location,Donut Shop,Diner,Discount Store,Dog Run,1
3,M1G,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Athletics & Sports,Hakka Restaurant,Fried Chicken Joint,Bakery,Lounge,Thai Restaurant,Caribbean Restaurant,Bank,Donut Shop,Dog Run,0


In [21]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

# create map
map_clusters = folium.Map(location = [latitude_tor, longitude_tor], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighbourhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [22]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
3,Scarborough,Coffee Shop,Korean Restaurant,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,0
4,Scarborough,Athletics & Sports,Hakka Restaurant,Fried Chicken Joint,Bakery,Lounge,Thai Restaurant,Caribbean Restaurant,Bank,Donut Shop,Dog Run,0
26,North York,Japanese Restaurant,Café,Gym / Fitness Center,Basketball Court,Caribbean Restaurant,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,0
58,Downtown Toronto,Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Bar,Gym,Bakery,Clothing Store,Hotel,0
90,Etobicoke,River,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,0
94,Etobicoke,Bank,Jewelry Store,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Diner,0
95,Etobicoke,Pizza Place,Liquor Store,Café,Shopping Plaza,Beer Store,Pharmacy,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,0
97,North York,Baseball Field,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,0
99,Etobicoke,Pizza Place,Chinese Restaurant,Intersection,Sandwich Place,Coffee Shop,Middle Eastern Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,0


In [23]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
2,Scarborough,Pizza Place,Medical Center,Electronics Store,Mexican Restaurant,Breakfast Spot,Rental Car Location,Donut Shop,Diner,Discount Store,Dog Run,1
13,Scarborough,Pizza Place,Thai Restaurant,Chinese Restaurant,Noodle House,Rental Car Location,Fried Chicken Joint,Italian Restaurant,Fast Food Restaurant,Gluten-free Restaurant,Curling Ice,1
17,North York,Mediterranean Restaurant,Pool,Dog Run,Golf Course,Athletics & Sports,Yoga Studio,Drugstore,Diner,Discount Store,Doner Restaurant,1
42,East Toronto,Park,Sandwich Place,Brewery,Food & Drink Shop,Steakhouse,Fish & Chips Shop,Fast Food Restaurant,Light Rail Station,Liquor Store,Board Shop,1
47,Central Toronto,Dessert Shop,Sandwich Place,Italian Restaurant,Café,Restaurant,Sushi Restaurant,Seafood Restaurant,Coffee Shop,Pizza Place,Thai Restaurant,1
60,Downtown Toronto,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gym,Deli / Bodega,Gastropub,Salad Place,Bar,1
64,Central Toronto,Jewelry Store,Sushi Restaurant,Park,Trail,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,1
67,Downtown Toronto,Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Bakery,Dumpling Restaurant,Chinese Restaurant,Mexican Restaurant,Dim Sum Restaurant,1
74,York,Park,Market,Women's Store,Fast Food Restaurant,Pharmacy,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,1
76,West Toronto,Bakery,Supermarket,Pharmacy,Discount Store,Pool,Café,Bank,Bar,Middle Eastern Restaurant,Music Venue,1


In [24]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Scarborough,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Health & Beauty Service,2
1,Scarborough,Golf Course,Bar,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio,2
5,Scarborough,Playground,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Gym,2
6,Scarborough,Discount Store,Train Station,Chinese Restaurant,Department Store,Coffee Shop,Convenience Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,2
7,Scarborough,Bus Line,Bakery,Fast Food Restaurant,Metro Station,Intersection,Bus Station,Soccer Field,Park,Electronics Store,Eastern European Restaurant,2
8,Scarborough,Intersection,Motel,American Restaurant,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,2
9,Scarborough,College Stadium,Café,General Entertainment,Skating Rink,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,2
10,Scarborough,Indian Restaurant,Thrift / Vintage Store,Pet Store,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Yoga Studio,Diner,Discount Store,Dog Run,2
11,Scarborough,Breakfast Spot,Shopping Mall,Sandwich Place,Bakery,Middle Eastern Restaurant,Auto Garage,Paper / Office Supplies Store,Smoke Shop,Electronics Store,Eastern European Restaurant,2
12,Scarborough,Skating Rink,Clothing Store,Lounge,Breakfast Spot,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,2


In [25]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
38,East York,Sporting Goods Shop,Coffee Shop,Grocery Store,Burger Joint,Bank,Beer Store,Smoothie Shop,Bike Shop,Shopping Mall,Breakfast Spot,3
43,East Toronto,Café,Coffee Shop,Gastropub,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Park,Brewery,Seafood Restaurant,3
57,Downtown Toronto,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Bar,Ice Cream Shop,Bubble Tea Shop,Falafel Restaurant,Middle Eastern Restaurant,3


In [26]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
82,West Toronto,Café,Mexican Restaurant,Flea Market,Bakery,Cajun / Creole Restaurant,Italian Restaurant,Fried Chicken Joint,Diner,Furniture / Home Store,Arts & Crafts Store,4
